In [ ]:
!pip install transformers datasets huggingface_hub --upgrade peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 76.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 419.0/419.0 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 17.9 MB/s eta 0:00:00
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.23.4
    Uninstalling huggingface-hub-0.23.4:
      Successfully uninstalled huggingface-hub-0.23.4
  Attempting uninstall: transformers
    Found existing installation: transformers 4.42.3
    Uninstalling transformers-4.42.3:
      Successfully uninstalled transformers-4.42.3


# Connect to distilbert-base-uncased from huggingface

In [ ]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline
# Import required libraries
from transformers import DistilBertForSequenceClassification, DistilBertTokenizerFast, Trainer, TrainingArguments, pipeline
from datasets import load_dataset, load_metric


# Load the pre-trained model and tokenizer
model_name = "distilbert-base-uncased"
model = DistilBertForSequenceClassification.from_pretrained(model_name, num_labels=2)
tokenizer = DistilBertTokenizerFast.from_pretrained(model_name)

2024-07-22 23:04:53.485844: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-22 23:04:53.485971: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-22 23:04:53.635004: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

# Find and Load SST-2 Dataset from Hugging Face to fine tune model

In [ ]:
from datasets import load_dataset
import pandas as pd
from tabulate import tabulate
# Load the SST-2 dataset
dataset = load_dataset('glue', 'sst2') #The sst2 (Stanford Sentiment Treebank 2) is one of the datasets within the GLUE benchmark. SST-2 specifically focuses on sentiment analysis, providing sentences labeled as either positive or negative



# Convert samples to DataFrames for better formatting
train_samples = pd.DataFrame(dataset['train'][:5])  # First 5 entries from the training set
validation_samples = pd.DataFrame(dataset['validation'][:5])  # First 5 entries from the validation set

# Function to display DataFrame as a well-formatted table
def display_table(title, df):
    print(f"\n{title}")
    print(tabulate(df, headers='keys', tablefmt='pretty', showindex=False))

# Display the sample entries as well-formatted tables with titles
display_table("Sample entries from the SST-2 training dataset:", train_samples)
display_table("Sample entries from the SST-2 validation dataset:", validation_samples)



# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples['sentence'], truncation=True, padding=True)

encoded_dataset = dataset.map(tokenize_function, batched=True)


Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]


Sample entries from the SST-2 training dataset:
+------------------------------------------------------------------------------------------+-------+-----+
|                                         sentence                                         | label | idx |
+------------------------------------------------------------------------------------------+-------+-----+
|                       hide new secretions from the parental units                        |   0   |  0  |
|                           contains no wit , only labored gags                            |   0   |  1  |
| that loves its characters and communicates something rather beautiful about human nature |   1   |  2  |
|                 remains utterly satisfied to remain the same throughout                  |   0   |  3  |
|         on the worst revenge-of-the-nerds clichés the filmmakers could dredge up         |   0   |  4  |
+------------------------------------------------------------------------------------------+---

Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

Map:   0%|          | 0/1821 [00:00<?, ? examples/s]

# Connect to Hugging Face

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

# Check if connected

In [ ]:
import requests

# URL to test (Hugging Face's model hub)
url_to_test = "https://huggingface.co"

try:
    # Send a GET request to the URL
    response = requests.get(url_to_test)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        print("Connected to Hugging Face's model hub.")
    else:
        print("Not connected to Hugging Face's model hub.")
except requests.exceptions.RequestException as e:
    print(f"Connection error: {e}")


Connected to Hugging Face's model hub.


# Run full fine tune, commit to Hugging Face (DO NOT RE RUN)

In [ ]:
import torch
from datasets import load_metric
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments
from huggingface_hub import HfApi, HfFolder

# Check if GPU is available and set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load the accuracy metric
metric = load_metric('accuracy')

# Define the compute_metrics function
def compute_metrics(p):
    predictions, labels = p
    predictions = predictions.argmax(axis=1)
    return metric.compute(predictions=predictions, references=labels)

# Set up training arguments
training_args = TrainingArguments(
    output_dir='./results', # Directory to save the model and checkpoints
    evaluation_strategy='epoch',# Evaluate the model at the end of each epoch
    save_strategy='epoch', # Save the model at the end of each epoch
    learning_rate=2e-5,# Learning rate for the optimizer
    per_device_train_batch_size=16, # Batch size for training on each device
    per_device_eval_batch_size=16,  # Batch size for evaluation on each device
    num_train_epochs=3, # Number of epochs to train the model
    weight_decay=0.01, # Weight decay to apply for regularization
    logging_dir='./logs', # Directory to save the logs
    logging_steps=10, # Log training information every 10 steps
    push_to_hub=True  # Enable pushing to hugging face
)

# Load the model and move it to the appropriate device
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")
model.to(device)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset['train'],
    eval_dataset=encoded_dataset['validation'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Evaluate before fine-tuning
print("Evaluating before fine-tuning...")
pre_finetune_results = trainer.evaluate()
print("Results before fine-tuning:", pre_finetune_results)

# Start training
trainer.train()

# Evaluate after fine-tuning
print("Evaluating after fine-tuning...")
post_finetune_results = trainer.evaluate()
print("Results after fine-tuning:", post_finetune_results)

# Save the model and tokenizer locally
model.save_pretrained('./fine-tuned-model')
tokenizer.save_pretrained('./fine-tuned-tokenizer')

# Push the model and tokenizer to Hugging Face's Model Hub
model.push_to_hub("my-fine-tuned-distilbert")
tokenizer.push_to_hub("my-fine-tuned-distilbert")

Using device: cuda


/opt/conda/lib/python3.10/site-packages/datasets/load.py:759: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.2/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
Y

Evaluating before fine-tuning...


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

...

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Results after fine-tuning: {'eval_loss': 0.3600134253501892, 'eval_accuracy': 0.9036697247706422, 'eval_runtime': 1.5224, 'eval_samples_per_second': 572.768, 'eval_steps_per_second': 18.392, 'epoch': 3.0}


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Benuehlinger/my-fine-tuned-distilbert/commit/23bdb76ee908593a411656a40b3441909162c5a9', commit_message='Upload tokenizer', commit_description='', oid='23bdb76ee908593a411656a40b3441909162c5a9', pr_url=None, pr_revision=None, pr_num=None)